In [1]:
import sys; sys.path.append('..')
from osp import *

In [2]:
df_meta = get_corpus_metadata()
ids_phil = df_meta[df_meta['discipline'] == 'Philosophy'].index.tolist()
ids_lit = df_meta[df_meta['discipline'] == 'Literature'].index.tolist()

In [8]:
import numpy as np

groups = [
    ('Philosophy', ids_phil),
    ('Literature', ids_lit),
]

def get_mdw_pos(ids1, ids2, name1="Group 1", name2="Group 2", feat_n=FEAT_N, feat_min_count=FEAT_MIN_COUNT, incl_deprel=True, incl_pos=True):
    df_pos_grp1 = get_pos_counts(ids1, incl_deprel=incl_deprel, incl_pos=incl_pos)
    df_pos_grp2 = get_pos_counts(ids2, incl_deprel=incl_deprel, incl_pos=incl_pos)

    words_grp1 = get_pos_word_counts(ids1)
    words_grp2 = get_pos_word_counts(ids2)

    egs_grp1 = get_pos_word_egs(ids1)
    egs_grp2 = get_pos_word_egs(ids2)


    df_pos = pd.concat([df_pos_grp1.assign(_target=name1), df_pos_grp2.assign(_target=name2)])
    fisher_results = fisher_test_pos(df_pos, target_col='_target', g1=name1, g2=name2)

    df_means = df_pos.groupby('_target').mean()
    feat2grp2mean = df_means.to_dict()
    df_sums = df_pos.groupby('_target').sum()

    ld = []
    feat1 = None

    colname1 = f'1 ({name1})'
    colname2 = f'2 ({name2})'

    for feat in feat2grp2mean:
        feat_d1 = {'feat':feat, 'group':name1}
        feat_d2 = {'feat':feat, 'group':name2}

        grp2mean = feat2grp2mean[feat]
        
        feat_d1[f'fpk'] = grp2mean[name1]
        feat_d2[f'fpk'] = grp2mean[name2]
        feat_d1[f'top'] = get_egs(words_grp1[feat], n=feat_n, min_count=feat_min_count)
        feat_d2[f'top'] = get_egs(words_grp2[feat], n=feat_n, min_count=feat_min_count)

        feat_d1[f'egs'] = get_egs(words_grp1[feat], n=feat_n, min_count=feat_min_count, word2eg=egs_grp1[feat])
        feat_d2[f'egs'] = get_egs(words_grp2[feat], n=feat_n, min_count=feat_min_count, word2eg=egs_grp2[feat])
        
        # feat1 = feat
        ld.extend([feat_d1, feat_d2])
    odf = pd.DataFrame(ld).dropna().set_index('feat')
    # diffcol = f'fpk1-fpk2'
    # odf[diffcol] = odf[f'fpk1'] - odf[f'fpk2']
    odf = fisher_results.join(odf).sort_values('odds_ratio', ascending=False)
    # odf['odds_ratio_log'] = np.log10(odf['odds_ratio'])
    # odf['odds_ratio_log_abs'] = np.abs(odf['odds_ratio_log'])
    odf['feat_desc'] = [POS2DESC.get(feat,'?') for feat in odf.index]
    odf = odf.reset_index()
    return odf.set_index(['feat','group'])
    # odf = odf[['feat', 'feat_desc','sum1','sum2','fpk1','fpk2','fpk1-fpk2','odds_ratio','top1','top2','egs1','egs2','p_value', 'sig']]
    # odf = odf.set_index(['feat','feat_desc'])
    # end_cols = ['p_value', 'sig']
    # odf = odf[[c for c in odf.columns if c not in end_cols] + end_cols]
    # return odf.rename(columns={
    #     'sum1': f'# {colname1}',
    #     'sum2': f'# {colname2}',
    #     'fpk1': f'#/K {colname1}',
    #     'fpk2': f'#/K {colname2}',
    # })


In [9]:
pd.options.display.max_colwidth = None
odf = get_mdw_pos(ids_phil, ids_lit, 'Philosophy', 'Literature', feat_n=10, feat_min_count=1, incl_deprel=False).round(2)
odf

sum1   sum2  pct1  pct2  odds_ratio  p_value  sig    fpk  \
feat group                                                                   
NFP  Philosophy    547     71  0.04  0.01        3.91      0.0  ***   0.33   
     Literature    547     71  0.04  0.01        3.91      0.0  ***   0.08   
LS   Philosophy    624     87  0.04  0.01        3.64      0.0  ***   0.38   
     Literature    624     87  0.04  0.01        3.64      0.0  ***   0.10   
FW   Philosophy   1553    223  0.10  0.03        3.54      0.0  ***   0.94   
...                ...    ...   ...   ...         ...      ...  ...    ...   
NNP  Literature  30557  37178  2.06  4.94        0.41      0.0  ***  43.53   
VBD  Philosophy   8911  12945  0.60  1.72        0.35      0.0  ***   5.40   
     Literature   8911  12945  0.60  1.72        0.35      0.0  ***  15.16   
NNPS Philosophy    952   1844  0.06  0.25        0.26      0.0  ***   0.58   
     Literature    952   1844  0.06  0.25        0.26      0.0  ***   2.16   

                                                                                                    top  \
feat group                                                                                                
NFP  Philosophy                                                                   * { ~ | - · -> > \ **   
     Literature                                                                  - * | { *** ;- > z ~ ^   
LS   Philosophy                                                             ii a i iii v k vi b iv xlvi   
     Literature                                                        ii a vi v i viii iii iv xli xxxi   
FW   Philosophy                                               i.e. e.g. etc i.e etc. e.g eds. cf pp. c.   
...                                                                                                 ...   
NNP  Literature                    shakespeare god english mr. england milton wordsworth new james john   
VBD  Philosophy                                        was were had did thought said saw made knew took   
     Literature                                       was had were did made came wrote said became took   
NNPS Philosophy  states nations americans greeks netherlands principles ages christians sciences essays   
     Literature        states americans ages works studies jews indians christians tales entertainments   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        egs  \
feat group                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
NFP  Philosophy                                                                                                              "G < / >(*****), we call physical" "Axiom P **{** ext (S)   S G" "estimate of p (**~** I / M) is still" "PF(q) cF **|** EH" "not freeze (**-** > £ —" "Statistical evidence **·** Epistemic impurism" "for interpreting **->** as a material conditional" "case that y **>** xx, where xx is" "Iu{H) - mod? **\**, we see that" "Both (i ******) the stick is"   
     Literature                                                                                                             "OF MORE - AND **-** LESS : LETTER" "in what manne

In [5]:
# pd.options.display.max_colwidth = None
# # pprint(dict(odf.loc['cop']))
# odf

In [6]:
ids1 = df_meta.query('discipline == "Philosophy" & 1920<=year<1970').index.tolist()
ids2 = df_meta.query('discipline == "Philosophy" & 2025>=year>=1970').index.tolist()
odf = get_mdw_pos(ids1, ids2, 'Early Philosophy', 'Late Philosophy', feat_n=10, feat_min_count=1).round(2)
odf[odf.sig=="***"]

,feat,sum1,sum2,pct1,pct2,odds_ratio,p_value,sig,group,fpk,top,egs,feat_desc
2,'',3475,5826,0.43,0.25,1.73,0.0,***,Early Philosophy,8.56,""" ' '' '. »","""by definition ** ** (Nature and Mind"" ""of existence ** ** - the phrase is"" ""agent : e.g., ** ** using a motor vehicle"" ""or things **"" ""sunflowers are yellow **»** are intelligible""",?
3,'',3475,5826,0.43,0.25,1.73,0.0,***,Late Philosophy,4.96,""" ' « »","""word pure ** ** when referring"" ""ext (S) ** ** S G is a partition"" ""be - doo **«** now will turn"" ""Connectives A, V, **»**. Parentheses""",?
4,``,3339,5760,0.41,0.25,1.68,0.0,***,Early Philosophy,8.22,""" ' « ''","""and about ** ** generic traits"" ""the previous ** ** vulgar measure"" ""sentences like **«** all sunflowers"" ""which remains ** ** gently optimistic""",?
5,``,3339,5760,0.41,0.25,1.68,0.0,***,Late Philosophy,4.91,""" ' » « ¬","""understand the word ** ** pure when referring"" ""fi, U and ** ** set theoretical"" ""I am uttering **»** shoo - be - doo"" ""doo - be doo **«** now , then"" ""truth to P and **¬** P but falsity to""",?
8,VBD,2888,5501,0.36,0.23,1.52,0.0,***,Early Philosophy,7.11,was were had did thought made saw said knew became,"""clearer what I **was** trying to say"" ""The Greeks **were** good at it, and"" ""Dewey decided he **had** been wrong in trying"" ""that, if we **did** not say this"" ""that I once **thought** scientific ethics"" ""In them Dewey **made** a conscious attempt"" ""conditions I **saw** only one spot"" ""puzzled by what I **said** or failed to say"" ""he not only **knew**, but also wanted"" ""others, it **became** increasingly clear""","Verb, past tense"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,NFP,50,472,0.01,0.02,0.31,0.0,***,Early Philosophy,0.12,* - *( ~ *. *) -* ;) =y >,"""concepts (Tarski *****)) or as the"" ""craftsmanship. **-** Mr. A. XXX"" ""this by A ***(** P)"" ""[boiXi => **~** CoiXi] in this"" ""prehension :"" ""form of logical ***)** A. Tarski : Introduction"" ""H (x) **-*** A(x)], we must"" ""character (?joo?"" ""the form x **=y** (or x > y"" ""or x **>** y or x <y""",Superfluous punctuation
185,NFP,50,472,0.01,0.02,0.31,0.0,***,Late Philosophy,0.40,* { ~ | · -> > \ ** ^,"""G < / >(*****), we call physical"" ""Axiom P **{** ext (S) S G"" ""estimate of p (**~** I / M) is still"" ""PF(q) cF **|** EH"" ""Statistical evidence **·** Epistemic impurism"" ""for interpreting **->** as a material conditional"" ""case that y **>** xx, where xx is"" ""Iu{H) - mod? **\**, we see that"" ""Both (i ******) the stick is"" ""ix) A] M **^**""",Superfluous punctuation
186,SYM,26,565,0.00,0.02,0.13,0.0,***,Early Philosophy,0.06,= / +,"""AC **=** Conscious states"" ""resources and **/** or values distributed"" ""ai+hirbai+ pramis **+** topei+yu+smiO+faiv+dalbrz""",Symbol
187,SYM,26,565,0.00,0.02,0.13,0.0,***,Late Philosophy,0.48,= / + { \ /- ^ // /( x,"""predicates, £ **=** { E, F,. Individual"" ""of sentences < **/** >v that are verifiable"" ""means, if K **+** is the next cardinal"" ""predicates, £ = **{** E, F,. Individual"" ""fte] |, **\** chairman\)) and"" ""difference in **/-** values of two point"" ""G t^x and x **^** t"" ""likewise acquire **//** - clauses as"" ""simply write **/(** C /) for it"" ""y) V (x **x** z) x x (y A z""",Symbol


In [7]:
ids1 = df_meta.query('discipline == "Literature" & 1920<=year<1970').index.tolist()
ids2 = df_meta.query('discipline == "Literature" & 2025>=year>=1970').index.tolist()
odf = get_mdw_pos(ids1, ids2, 'Early Literature', 'Late Literature', feat_n=10, feat_min_count=1).round(2)
odf[odf.sig=="***"]

,feat,sum1,sum2,pct1,pct2,odds_ratio,p_value,sig,group,fpk,top,egs,feat_desc
0,csubj:pass,33,20,0.01,0.00,3.81,0.0,***,Early Literature,0.14,extension writing made come provide indicates led used superseded spelled,"""system was an **extension** of the worship"" ""Writing** and speaking should"" ""play had already **made** use of the Standards"" ""not themselves **come** in with dancing"" ""of analogy can **provide** the key to"" ""Paradise as a place **indicates** the fading in Adam"" ""themselves had **led** the way and were"" ""word is not **used** when there is a"" ""not entirely **superseded** the earlier use"" ""nearly always **spelled** in full, and that""",?
1,csubj:pass,33,20,0.01,0.00,3.81,0.0,***,Late Literature,0.04,used done impaired establishing have threatened thing act seems promise,"""cognates - are **used** extensively as"" ""done, have **done** a good, good thing"" ""individual was **impaired** in his or her"" ""Indeed **establishing** the metaphoric"" ""language, **have** a potential for"" ""society could be **threatened**, not just by rising"" ""the furthest **thing** from the minds"" ""but a simple **act** of transmission"" ""problem **seems** appropriate to"" ""is the latent **promise** of a twentieth""",?
2,LS,40,34,0.01,0.00,2.72,0.0,***,Early Literature,0.16,ii a vi v i viii iii xli xxxi xi,"""second Sunday (**ii"" ""A**. In fact Jaggard"" ""VI"" ""V"" ""following : (**i**) Therefore see"" ""I. **viii**) attack on Pantalobus"" ""take effect (**iII"" ""XLI**. XLI"" ""XXXI"" ""XI""",List item marker
3,LS,40,34,0.01,0.00,2.72,0.0,***,Late Literature,0.06,ii vi v i a iii iv vii viii,"""II**. READING THE"" ""VI"" ""V"" ""above, note **i"" ""A**. The exact relation"" ""others (v. **iii"" ""Vandals (**IV"" ""VII"" ""vnI, and **vIIi**) greatly widen""",List item marker
4,NFP,36,31,0.01,0.00,2.68,0.0,***,Early Literature,0.15,- * ;- { | > z ~~,"""Tintern Abbey : **-** If this Be but"" ""front inalte ***** rable, Contemple"" ""than formerly **;-** the rage of retiring"" ""and studies **{** Disc"" ""en castellano **|** por I A. R. y R"" ""alternating with **>** Leireso > or"" ""later writers. **z"" ""than discreet;""",Superfluous punctuation
...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,-LRB-,740,2194,0.15,0.20,0.78,0.0,***,Late Literature,3.91,( [ <,"""overcome them **(** for we tend to"" ""that preclude **[** s] voyeuristic"" ""in Province > **<** Paris : topographies""",Left parenthesis
176,compound,2894,8694,0.59,0.77,0.77,0.0,***,Early Literature,11.91,self century a twenty manuscript prose love country stage london,"""are not always **self** - consistent individuals"" ""eighteenth - **century**, middle - class"" ""tendency of the **A** man to compose"" ""Only **twenty** - seven Names make"" ""The **manuscript** version of the"" ""some irregular **prose** lines with initial"" ""The **love** psychology, derived"" ""As also a **Country** Dialogue by Mrs"" ""than that Ariel **stage** - managed the show"" ""some of the **London** Hospitals, challenged""",?
177,compound,2894,8694,0.59,0.77,0.77,0.0,***,Late Literature,15.50,self century world class art material living reading family time,"""balances fetishistic **self** - interest about"" ""nineteenth - **century** Catholicism, last"" ""to the nearly **world** - historical"" ""use of lower - **class** disguise as a means"" ""thee ther thou **art** nat receyved"" ""of the very **material** effects that fashion"" ""part of the **living** body, good and"" ""realist modes of **reading** race"" ""part of her **family** history — both"" ""canvas of space - **time** superimposition""",?
182,-RRB-,605,1936,0.12,0.17,0.72,0.0,***,Early Literature,2.49,) ] >,"""as elsewhere **)** are the prevalence"" ""with [them **]**, they were upon"" ""with > Leireso **>** or > Leieso > Loiesoz""",Right parenthesis
